# Homework 5
### Joe Schneider, Kiran Ferrini, Mark Armstrong

# Question 1

The five elements of dynamic programming are:

1. The State Space:  $𝑋 = \{\text{Bad Year,  Normal Year,  Good Year}\}$
2. The Control Set:  $𝑈 =  \{\text{Ad Buy,  No Ad Buy}\}$
3. The Reward Function:  $𝑟(𝑥,𝑢)$

>{Good Year, Ad Buy} $\Rightarrow$  \\$3 Million  
{Good Year, No Ad Buy} $\Rightarrow$ \\$4 Million    
{Normal Year, Ad Buy} $\Rightarrow$  \\$1 Million  
{Normal Year, No Ad Buy} $\Rightarrow$ \\$2 Million   
{Bad Year, Ad Buy} $\Rightarrow$ \\$0  
{Bad Year, No Ad Buy} $\Rightarrow$ \\$1 Million  

4. A Stochastic Transition Rule for the Distribution of Next Period's State:  $𝑥_{𝑡+1}∼𝑔(𝑥_𝑡,𝑢_𝑡)$

>$\begin{aligned}
M(\text{No Ad Buy}) = 
\begin{bmatrix}
0.8 & 0.2 & 0\\
0.2 & 0.8 & 0 \\
0.1 & 0.4 & 0.5
\end{bmatrix}
\;\;\;\;\;\; \text{and} \;\;\;\;\;\;
M(\text{Ad Buy}) = 
\begin{bmatrix}
0.5 & 0.5 & 0\\
0.1 & 0.6 & 0.3 \\
0.1 & 0.3 & 0.6
\end{bmatrix}\end{aligned}$
 
5. The Discount Factor: 𝛽∈[0,1)

>$\begin{aligned}
\beta = \frac{1}{1+r}
\end{aligned}$

# Question 2

Bellman Equation:

>$\begin{aligned}
V(x_t) = \max_{u_t \in U} \;\; {r(x_t,\;u_t)+\beta \cdot E [\;V(x_{t+1})\;|\;x_t]}
\end{aligned}
$

For the company in a **Normal Year**:

>$\begin{aligned}
V(\text{Normal}) = \max \;\; \{\;2 \;+\;  \frac{1}{1+r} \; \cdot \; [0.2 \;\cdot\; V(\text{Bad}) \;+\; 0.8 \; \cdot \;  V(\text{Normal})], \;\; 
1 \;+\; \frac{1}{1+r} \;\cdot\; [0.1 \;\cdot\; V(\text{Bad}) \; + \; 0.6 \;\cdot\; V(\text{Normal}) \; + \; 0.3 \; \cdot \; V(\text{Good})]  \;\}
\end{aligned}$

# Question 3

In [4]:
import numpy as np
import pandas as pd
import quantecon as qe

In [5]:
states = ['Bad', 'Normal', 'Good']
controls = ['No Buy', 'Buy']
reward= np.array([1,0,2,1,4,3]).reshape(3,2)
beta = 0.9
value = np.zeros(len(states))

# Question 4

In [6]:
M_nobuy = np.array([0.8,0.2,0,0.2,0.8,0,0.1,0.4,0.5]).reshape(3,3)
M_buy = np.array([0.5,0.5,0,0.1,0.6,0.3,0.1,0.3,0.6]).reshape(3,3)
transitions = {'No Buy':M_nobuy, 'Buy':M_buy}

# Question 5

In [10]:
def check_params(states, controls, reward, beta, value, transitions):
    n = len(states)
    m = len(controls)
    if reward.shape != (n, m):
        print('The reward function has the wrong dimensions.')
    if len(value) != n:
        print('The value function is not the right length')
    if len(transitions) != m:
        print("There is not the requisite number of transition matrices.")
    if not all(transitions[control].shape == (n,n) for control in controls):
        print("At least one of the Markov transitions is one of the wrong dimension.")
    if not all(np.ones(n)@transitions[control]@np.ones(n) == n for control in controls):
        print("One of the transitions matrices is not Markov")
    if not all((transitions[control] >= 0).all() and (transitions[control] <= 1).all() for control in controls):
        print("The elements of the transition matrices are not probabilities.")
    if beta < 0 or beta >= 1:
        print("Something is wrong with the discount factor.")
    return (print("Requisite checks done, all is compatible."))

In [11]:
check_params(states, controls, reward, beta, value, transitions)

Requisite checks done, all is compatible.


# Question 6

In [14]:
def transition_probs(state, control, transitions):
    row = states.index(state)
    tmat = transitions[control]
    return tmat[row]

In [15]:
transition_probs('Normal', 'Buy', transitions)

array([0.1, 0.6, 0.3])

# Question 7

In [16]:
def update_value(states, controls, reward, transitions, value, beta):
    val_copy = value.copy()
    for state in states:
        val = []
        for control in controls:
            payoff = reward[states.index(state), controls.index(control)] + beta * (value @ transition_probs(state, control, transitions))
            val.append(payoff)
        val_copy[states.index(state)] = np.max(val)
    return(val_copy)

In [17]:
update_value(states, controls, reward, transitions, value, beta)

array([1., 2., 4.])

# Question 8

In [18]:
def solve(states,controls,reward,transitions,value,beta):
    epsilon = 1e-6
    value_check = value.copy()
    max_iter = 500
    counter = 0
    
    while (all(abs(update_value(states, controls, reward, transitions, value_check, beta) - value_check) >= epsilon)) and (counter <= max_iter):
        value_check = update_value(states, controls, reward, transitions, value_check, beta)
        counter += 1
    if counter >=500:
        print('Did not converge!')
    return(value_check)

In [19]:
value = solve(states,controls,reward,transitions,value,beta)
value

array([14.80552819, 17.4752711 , 21.13380769])

# Question 9

In [22]:
def optimal_policy(states, controls, reward, transitions, value, beta):
    optimal = []
    for state in states:
        val = []
        for control in controls:
            payoff = reward[states.index(state), controls.index(control)] + beta * (value @ transition_probs(state, control, transitions))
            val.append(payoff)
        optimal.append(controls[np.argmax(val)])
    return(optimal)

In [23]:
op = optimal_policy(states, controls, reward, transitions, value, beta)
op

['No Buy', 'Buy', 'No Buy']

# Question 10

In [24]:
def transition_matrix_optimal(states, controls, reward, transitions, value, beta):
    op = optimal_policy(states, controls, reward, transitions, value, beta)
    op_transition_matrix = np.zeros(9).reshape(3,3)
    for control in range(len(op)):
        op_transition_matrix[control] = transitions[op[control]][control]
    return op_transition_matrix

In [25]:
P = transition_matrix_optimal(states, controls, reward, transitions, value, beta)
P

array([[0.8, 0.2, 0. ],
       [0.1, 0.6, 0.3],
       [0.1, 0.4, 0.5]])

In [26]:
mc = qe.MarkovChain(P)
p_stationary = mc.stationary_distributions 
p_stationary

array([[0.33333333, 0.41666667, 0.25      ]])

# Question 11

In [90]:
pd.DataFrame({'Reward if No Buy': list(reward[:, 0]), 'Reward if Buy': list(reward[:,1]), 'Solved Value Function': list(np.around(value, 2)), 
              'Optimal Policy': op, 'Probabilities from Ergodic Distribution': list(np.around(p_stationary[0],2))}, index = states).style.format(
                precision=2).set_table_styles([{'selector': 'th.col_heading', 'props': 'text-align: center;'},
                                                {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},], overwrite=False) 

,Reward if No Buy,Reward if Buy,Solved Value Function,Optimal Policy,Probabilities from Ergodic Distribution
Bad,1,0,14.81,No Buy,0.33
Normal,2,1,17.48,Buy,0.42
Good,4,3,21.13,No Buy,0.25


# Question 12
Creating the class:

In [110]:
class DP_Markov():
    
    def __init__(self, states, controls, reward, beta, value, transitions): #eliminated the need for the check_parameters by including it in the initialization
        import numpy as np
        import pandas as pd
        import quantecon as qe
        self.states = states
        self.controls = controls
        self.reward = reward
        self.beta = beta
        self.value = value
        self.transitions = transitions
        n = len(self.states)
        m = len(self.controls)
        if self.reward.shape != (n, m):
            raise Exception("Reward Function has incorrect dimensions")
        elif len(self.value) != n:
            raise Exception("Value Function is not the correct length")
        elif len(self.transitions) != m:
            raise Exception("Incorrect number of transition matrices")
        elif not all(self.transitions[control].shape == (n,n) for control in self.controls):
            raise Exception("At least one of the Markov transitions is not the correct dimension")
        elif not all(np.ones(n)@self.transitions[control]@np.ones(n) == n for control in self.controls):
            raise Exception("At least one of the transitions is not in Markov form")
        elif not all((self.transitions[control] >= 0).all() and (self.transitions[control] <= 1).all() for control in self.controls):
            raise Exception("At least one elements of the transition matrices are not probabilities between 0 and 1")
        elif self.beta < 0 or self.beta >= 1:
            raise Exception("The Disxcount factor is not in (0,1]")
        else:
            pass
    
    def __str__(self):
        return "States: " f'{self.states}' "\n Controls: " f'{self.controls}' "\n Reward: " f'{self.reward}' "\n Beta: " f'{self.beta}' "\n Value Function: " f'{self.value}' "\n Transitions: " f'{self.transitions}' 
    
    def transition_probs(self, state, control):
        row = self.states.index(state)
        tmat = self.transitions[control]
        return tmat[row]
    
    def update_value(self):
        val_copy = self.value.copy()
        for state in self.states:
            val = []
            for control in self.controls:
                payoff = self.reward[self.states.index(state), self.controls.index(control)] + self.beta * (self.value @ self.transition_probs(state, control))
                val.append(payoff)
            val_copy[self.states.index(state)] = np.max(val)
        return(val_copy)
    
    def solve(self):
        epsilon = 1e-6
        max_iter = 500
        counter = 0

        while (all(abs(self.update_value() - self.value) >= epsilon)) and (counter <= max_iter):
            self.value = self.update_value()
            counter += 1
        if counter >=500:
            print('Did not converge after 500 iterations')
        return(self.value)
    
    def optimal_policy(self):
        optimal = []
        for state in self.states:
            val = []
            for control in self.controls:
                payoff = self.reward[self.states.index(state), self.controls.index(control)] + self.beta * (self.value @ self.transition_probs(state, control))
                val.append(payoff)
            optimal.append(self.controls[np.argmax(val)])
        return(optimal)
    
    def transition_matrix_optimal(self):
        op = self.optimal_policy()
        op_transition_matrix = np.zeros(9).reshape(3,3)
        for control in range(len(op)):
            op_transition_matrix[control] = self.transitions[op[control]][control]
        return op_transition_matrix
    
    def optimal_ergodic_dist(self):
        P = self.transition_matrix_optimal()
        mc = qe.MarkovChain(P)
        p_stationary = mc.stationary_distributions 
        return p_stationary
    
    def summary_df(self):
        df = pd.DataFrame({'Reward if No Buy': list(self.reward[:, 0]), 'Reward if Buy': list(self.reward[:,1]), 'Solved Value Function': list(np.around(self.value, 2)), 'Optimal Policy': self.optimal_policy(), 'Probabilities from Ergodic Distribution': list(np.around(self.optimal_ergodic_dist()[0],2))}, index = states)
        return df.style.format(precision=2).set_table_styles([{'selector': 'th.col_heading', 'props': 'text-align: center;'},
                                                             {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},], overwrite=False) 

Defining the Parameters:

In [118]:
states = ['Bad', 'Normal', 'Good']
controls = ['No Buy', 'Buy']
reward= np.array([1,0,2,1,4,3]).reshape(3,2)
beta = 0.9
value = np.zeros(len(states))

M_nobuy = np.array([0.8,0.2,0,0.2,0.8,0,0.1,0.4,0.5]).reshape(3,3)
M_buy = np.array([0.5,0.5,0,0.1,0.6,0.3,0.1,0.3,0.6]).reshape(3,3)
transitions = {'No Buy':M_nobuy, 'Buy':M_buy}

Example of Failure Initialization:

In [119]:
transitionsbad = {'No Buy':M_nobuy}
DPclassfail = DP_Markov(states, controls, reward, beta, value, transitionsbad)

Exception: Incorrect number of transition matrices

Proper Initialization and Display of Parameters:

In [117]:
DPclass = DP_Markov(states, controls, reward, beta, value, transitions)
print(DPclass)

States: ['Bad', 'Normal', 'Good']
 Controls: ['No Buy', 'Buy']
 Reward: [[1 0]
 [2 1]
 [4 3]]
 Beta: 0.9
 Value Function: [0. 0. 0.]
 Transitions: {'No Buy': array([[0.8, 0.2, 0. ],
       [0.2, 0.8, 0. ],
       [0.1, 0.4, 0.5]]), 'Buy': array([[0.5, 0.5, 0. ],
       [0.1, 0.6, 0.3],
       [0.1, 0.3, 0.6]])}


Transition Probabilities Method:

In [83]:
DPclass.transition_probs('Good', 'No Buy')

array([0.1, 0.4, 0.5])

Update Value Function Method:

In [84]:
DPclass.update_value()

array([14.80552909, 17.47527201, 21.13380859])

Solving the Dynamic Program Method:

In [85]:
DPclass.solve()

array([14.80552819, 17.4752711 , 21.13380769])

Optimal Policy Method:

In [86]:
DPclass.optimal_policy()

['No Buy', 'Buy', 'No Buy']

Optimal Transition Matrix Method:

In [87]:
DPclass.transition_matrix_optimal()

array([[0.8, 0.2, 0. ],
       [0.1, 0.6, 0.3],
       [0.1, 0.4, 0.5]])

Optimal Ergodic Distribution Method:

In [88]:
DPclass.optimal_ergodic_dist()

array([[0.33333333, 0.41666667, 0.25      ]])

Summary Table Method:

In [89]:
DPclass.summary_df()

,Reward if No Buy,Reward if Buy,Solved Value Function,Optimal Policy,Probabilities from Ergodic Distribution
Bad,1,0,14.81,No Buy,0.33
Normal,2,1,17.48,Buy,0.42
Good,4,3,21.13,No Buy,0.25
